<a href="https://colab.research.google.com/github/FranciscoOcampoPredictiva/azureml_course/blob/main/Python_Script_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Installation and Setup

In [1]:
# Importing the classes
from azureml.core import Workspace, Experiment, Run

ModuleNotFoundError: ignored

In [2]:
# Accessing the workspace
ws = Workspace.from_config(path='/content/config.json')

NameError: ignored

In [3]:
# Getting the context of the experiment run
new_run = Run.get_context()

NameError: ignored

# Part 2: Data Processing

## Importing the libraries and dataset



In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Loading the dataset
dataset = pd.read_csv('/content/Churn_Modelling.csv')

## Dealing with the missing data

In [ ]:
dataset.isnull().values.any()

## Encoding the categorical variables

In [ ]:
dataset = dataset.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [ ]:
dataset.select_dtypes(include='object').columns

In [ ]:
dataset = pd.get_dummies(dataset, drop_first=True)

## Splitting the dataset into train and test set

In [ ]:
X = dataset.drop(columns='Exited')
Y = dataset['Exited']

In [ ]:
from sklearn.model_selection import train_test_split
x_test, x_train, y_test, y_train = train_test_split(X, Y, test_size=0.2, random_state=0)

# Part 3:Training and evaluating the ML Model

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0)
classifier.fit(x_train, y_train)

In [ ]:
# predicting the results on test set
y_pred = classifier.predict(x_test)

In [ ]:
# predicting the probabilities
y_prob = classifier.predict_proba(x_test)[:, 1]

In [ ]:
# getting the confusion matrix and the accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

In [ ]:
cm_dict = {
       "schema_type": "confusion_matrix",
       "schema_version": "1.0.0",
       "data": {
           "class_labels": ["N", "Y"],
           "matrix": cm.tolist()
       }
   }

# Part 4: Logging the values in experiment run

In [ ]:
new_run.log_confusion_matrix('Confusion Martix', cm_dict)
new_run.log('Accuracy Score', acc)

# Part 5: Creating the final dataset with predicted values and probabilities

In [ ]:
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

y_prob_df = pd.DataFrame(y_prob, columns=['Scored probabilities'])
y_pred_df = pd.DataFrame(y_pred, columns=['Scored Lebels'])

# create the final dataset
scored_dataset = pd.concat([x_test, y_test, y_pred_df, y_prob_df], axis=1)

In [ ]:
# Upload the dataset
scored_dataset.to_csv('./outputs/Churn_Scored.csv', index=False)

In [ ]:
# Complete the experiment run
new_run.complete()